In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np


In [3]:
from dspy.api import get_api
from dspy.api.bybit import Config

# <font color="grey">ByBit API</font>

In [4]:
# Fixed parameters
API_KEY = "StMMBpVlRHbiZF5tki"
API_SECRET = "DOe9UXJgeWLpTKWBibHBpjIDW0UpSON9FuUp"
SYMBOL = 'BTCUSDT'
COIN = 'USDT'

# Parameters to be optimized
EQUITY = 2.0
RANGE = 0.005
NUM_ORDERS = 5
POLLING_RATE = 1
TP_DIST = 0.0005
STOP_DIST = 0 #0.025

config = Config(API_KEY, 
                API_SECRET, 
                SYMBOL, 
                COIN, 
                EQUITY, 
                RANGE, 
                NUM_ORDERS, 
                POLLING_RATE, 
                TP_DIST, 
                STOP_DIST)

In [12]:
api = get_api('bybit', config=config)

In [13]:
symbols = ['BTCUSDT', 'ETHUSDT']

###  <a id='chapter1'> <font color="grey">Prices and Volumes</font></a>

In [14]:
mid = api.get_mid(symbols[1])
print(f"Last price: {mid}")

Last price:  2220.98


In [18]:
ask = api.get_ask(symbols[0])
bid = api.get_bid(symbols[0])
print(f"Bid: {bid}")
print(f"Ask: {ask}")

Bid:  [85871.8, 3.761]
Ask:  [85871.9, 6.073]


In [20]:
orderbook = api.get_orderbook(symbols[0], depth=5)
print(f"Bids: {orderbook['b']}")
print(f"Asks: {orderbook['a']}")

Bids: [[8.58546e+04 8.70400e+00]
 [8.58545e+04 3.49000e-01]
 [8.58540e+04 7.20000e-02]
 [8.58536e+04 4.90000e-02]
 [8.58535e+04 4.30000e-02]]
Asks: [[8.58547e+04 4.80000e-02]
 [8.58549e+04 1.00000e+01]
 [8.58560e+04 2.00000e-03]
 [8.58569e+04 2.00000e-03]
 [8.58578e+04 2.00000e-03]]


In [22]:
print(f"Latency: {api.get_latency(symbols[1])}")

Latency: 2


### Account info

In [25]:
balance = api.get_wallet_balance()
print(f"Balance: {balance}")

Balance: 2.33e-05


In [26]:
fees = api.get_fees('BTCUSDT')
print(f"Taker fees: {fees[0]}")
print(f"Maker fees: {fees[1]}")

Taker fees: 0.00055
Maker fees: 0.0002


### Position info

In [ ]:
pos = api.get_position(symbols[0])
pos is None

### Trading

In [ ]:
# Place market order
res = s.place_order('BTCUSDT', 0.005)
res

In [ ]:
balance = s.get_wallet_balance()
print("Balance: ", balance)

In [ ]:
pos = s.get_position(symbols[0])
pos

In [ ]:
entry = pos['entry_price']
entry

In [ ]:
entry*float(fees[0])*0.005

In [ ]:
liq_price = s.get_bid('BTCUSDT')[0]
liq_price-entry

In [ ]:
pos = s.get_positions(symbols)['BTCUSDT']
diff = (pos['mark_price']-pos['entry_price'])*0.005
diff2 = pos['unrealized_pnl']

In [ ]:
(diff-diff2)<1e-8

In [ ]:
(pos['realized_pnl']+fees[0]*pos['entry_price']*0.005)<1e-8

In [ ]:
# Close position
response = s.close_positions(['BTCUSDT'])
response

In [ ]:
# Limit orders
price = s.get_mid(config.symbol)
response = s.place_order(config.symbol,
                         -0.002,
                         price+10,
                         type='Limit')
print(response)

In [ ]:
price = s.get_mid(symbols[0])
response = s.place_order(symbols[0],
                         -0.001,
                         price+100,
                         type='Limit')
print(response)

In [ ]:
response = s.replace_order(symbols[0], response['order_id'], -0.001, price+100)
print(response)

In [ ]:
resp = s.cancel_order(symbols[0], response['order_id'])
print(resp)

In [ ]:
# Batch orders
response = s.place_batch_order(symbols[0], [-0.001, -0.001], [price+50, price+100])

In [ ]:
s.cancel_batch_order(symbols[0], [x['order_id'] for x in response])

In [ ]:
s.cancel_all_orders([symbols[0]])

In [ ]:
s.close_positions(['BTCUSDT'])

In [ ]:
mm(s, config)

In [ ]:
s.cancel_all_orders(config.symbol)
last = s.get_mid(config.symbol)
price_range = round(config.range * last, 2)
balance = s.get_wallet_balance()
available_equity = (balance * config.equity) / last
# Arithmetic grid
nn = 2*available_equity / (config.num_orders * (config.num_orders + 1))
long_qtys = np.round([nn * i for i in reversed(range(1, config.num_orders + 1))], 
                             decimals=3)
short_qtys = np.round([-i for i in long_qtys][::-1], decimals=3)
long_prices = np.round(
            np.linspace(last - price_range/2, 
                        last - nn, 
                        config.num_orders), 
            decimals=1
            )
short_prices = np.round(
            np.linspace(last + nn, 
                        last + price_range/2, 
                        config.num_orders), 
            decimals=1
            )


In [ ]:
long_order_ids = s.place_batch_order(config.symbol, long_qtys, long_prices)
short_order_ids = s.place_batch_order(config.symbol, short_qtys, short_prices)

In [ ]:
s.cancel_all_orders(config.symbol)

In [ ]:
mm(s, config)

In [ ]:
p = s.get_position(config.symbol)
p

In [ ]:
int(p['mark_price']*(1+0.005))

In [ ]:
np.linspace(0,1,10)